# Introduction to optimal interpolation and variational analysis

Alexander Barth, Aida Alvera Azcárate, Pascal Joassin, Jean-Marie Beckers, Charles Troupin.

Available under the [Creative Commons Attribution-ShareAlike 4.0](https://creativecommons.org/licenses/by-sa/4.0/) license.
$\newcommand{\vect}[1] {{\mathbf #1}}$ 

## The gridding problem
Gridding is the determination of a field $\phi(\vect{r})$, on a regular
grid of positions $\vect{r}$ based on arbitrarily located
observations.  Often the vector $\vect{r}$ is on a 2D, 3D or even 4D space.

<img src="./Images/oi_used_data.png" style="width: 300px; display: inline"> $\rightarrow$
<img src="./Images/oi_field.png" style="width: 300px; display: inline">

* In oceanography, _in situ_ observations are sparse, and the fewer observations are available, the harder the gridding problem is.
* Observations are inhomogeneously distributed in space and time (more observations in the coastal zones and in summer).
* The variability of the ocean is the sum of various processes occurring at different spatial and temporal scales. 

In [ ]:
using NCDatasets
using Makie, CairoMakie
using Statistics
using JupyterFormatter

## Load the data
Stored in the files `dan_field_obs.nc`are the following variables:
* `f`: the (full) field
* `F`: the observations (without noise)
* `Fe`: the observations (with noise)
* `X`, `Y`: the location of the observations
* `x`, `y`: the grid point of the field `f`
* `mask`: land-sea mask (true corresponds to sea and false to land)
* `pm`: the inverse of the resolution in the `x` direction
* `pn`: the inverse of the resolution in the `y` direction

In [ ]:
download_check(OIfile, OIfileURL)

In [ ]:
ds = Dataset(OIfile)
# load field
f = ds["f"].var[:, :]
x = ds["x"].var[:, :]
y = ds["y"].var[:, :]
mask = ds["mask"].var[:, :] .== 1

# load observations
F = ds["F"].var[:]
Fe = ds["Fe"].var[:]
X = ds["X"].var[:]
Y = ds["Y"].var[:]
close(ds);

## Create the metrics

In [ ]:
pm = ones(size(mask)) * 5;
pn = ones(size(mask)) * 5;

@show size(x);

In [ ]:
f2 = copy(f);
f2[.!mask] .= NaN;

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], aspect = DataAspect())
hm = heatmap!(ax, f2, colormap = :jet)
Colorbar(fig[1, 2], hm)
colgap!(fig.layout, -60)
fig

(Hypothetical) example of oceanographic field.

* This figure shows an idealized square domain with a barrier (_e.g._ a peninsula or a dike).
* This field is the true field  that we want to reconstruct based on observations. Let's assume that the field represents temperature. 
* The barrier suppresses the exchanges between each side of it.
* The field varies smoothly over some length-scale.

We define plotting functions that we will reused later

In [ ]:
function plotfield(f2; vmin = 16.5, vmax = 22, figtitle = "")

    f2 = copy(f2)
    f2[.!mask] .= NaN
    fig = Figure()
    ax = Axis(fig[1, 1], title = figtitle, aspect = DataAspect())
    co = contourf!(
        x[:, 1],
        y[1, :],
        f2,
        levels = range(vmin, vmax, length = 20),
        colormap = :jet,
    )
    Colorbar(fig[1, 2], co, label = "(°C)", vertical = true, labelrotation = 0)
    plot!(X, Y, color = :black)
    colgap!(fig.layout, -60)
    display(fig)
    return nothing
end

Plot the field and compare to the true solution

In [ ]:
function plotfieldcmp(f2, f; vmin = 16.5, vmax = 22, figtitle = "")

    fig = Figure(size = (800, 400))
    ax1 = Axis(fig[1, 1], title = "True field", aspect = DataAspect())
    co = contourf!(
        ax1,
        x[:, 1],
        y[1, :],
        f2,
        levels = range(vmin, vmax, length = 20),
        colormap = :jet,
    )
    Colorbar(fig[1, 2], co, label = "(°C)", labelrotation = 0)
    plot!(X, Y, color = :black)

    ax2 = Axis(fig[1, 3], title = figtitle, aspect = DataAspect())
    co2 = contourf!(
        ax2,
        x[:, 1],
        y[1, :],
        f,
        levels = range(vmin, vmax, length = 20),
        colormap = :jet,
    )
    Colorbar(fig[1, 4], co2, label = "(°C)", labelrotation = 0)
    plot!(X, Y, color = :black)

    display(fig)
end

In [ ]:
plotfield(f, figtitle = "Field with the location of the observations")

* In regions where a measurement campaign has been carried out, a higher spatial coverage is achieved. 
* Large gaps are also present. 
* Based on the value of the field at the shown location, we will estimate the true field.


Values of the true field extracted at the location of the observations

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], aspect = DataAspect())
contourf!(ax, x[:, 1], y[1, :], mask; levels = [0, 0.5], colormap = Reverse(:binary))
sc = scatter!(
    ax,
    X,
    Y,
    color = F,
    markersize = 10,
    colormap = :jet,
    colorrange = [16.5, 22.0],
)
xlims!(ax, 1, 10)
ylims!(ax, 1, 10)
Colorbar(fig[1, 2], sc, label = "(°C)", labelrotation = 0)
colgap!(fig.layout, -60)
fig

* some information about the position of the
structures and fronts is lost 
* no method can provide exactly the true field. But the
more information about its structure we include in the
analysis, the closer we can get to the true field.


## Observation errors

Observations are in general affected by different error sources and
other _problems_ that need to be taken into account:

- *Instrumental errors*: limited precision or possible bias of the sensor.
- *Representativeness errors*: the observations do not necessarily
  corresponds to the field we want to obtain. For example, we want to
  obtain a monthly average, but the observations are instantaneous (or
  averages over a very short period of time).
- *Synopticity errors*: all observations are not taken at the same time.
- Other error sources: human errors (e.g. permutation of
  longitude and latitude), transmission errors, malfunctioning of the
  instrument, wrong decimal separators...

Quality control is an important step to exclude suspicious data from
the analysis. But since this is a subjective decision, the data should
never be deleted but *flagged* as suspicious or bad data.

Let's plot the observations with noise (`Fe`):

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], aspect = DataAspect())
contourf!(ax, x[:, 1], y[1, :], mask; levels = [0, 0.5], colormap = Reverse(:binary))
sc = scatter!(
    ax,
    X,
    Y,
    color = Fe,
    markersize = 10,
    colormap = :jet,
    colorrange = [16.5, 22.0],
)
xlims!(ax, 1, 10)
ylims!(ax, 1, 10)
Colorbar(fig[1, 2], sc, label = "(°C)", labelrotation = 0)
colgap!(fig.layout, -60)
fig

* A random perturbation was added to the observations.
* This simulates the impact of the different error sources. 
* To simplify matters, each observation was perturbed independently.

## Data analysis and gridding methodologies

Because observations have errors, it is always better to produce a
field approximation and never a strict interpolation (left panel). 

<img src="Images/dan_griddata.png" style="width: 400px; display: inline" >
<img src="Images/dan_field.png" style="width: 400px; display: inline" >


* Gridded field using linear interpolation. This method is
  implemented in the function `griddata` of MATLAB and GNU Octave. Right panel shows true field.
  
* This figure shows what would happen if the observations would have been interpolated linearly. 
* The domain is decomposed into triangles where the vertices are the location of the data points based on the Delaunay triangulation. 
* Within each triangle, the value is interpolated linearly. 
* The noise on the observations is in general amplified when higher order schemes, such as cubic interpolation, are used.


There are a number of methods that have been traditionally used to
solve the gridding problem:

* Subjective: drawing by hand
* Objective: predefined mathematical operations


<img src="./Images/tropical_storm_claudette.jpg" style="width: 600px" >

Isohyet (lines of constant precipitation) drawn by hand (from http://www.srh.noaa.gov/hgx/hurricanes/1970s.htm)



## Objective analysis

As opposed to the subjective method, objective analysis techniques aim
to use mathematical formulations to infer the value of the field at
unobserved locations based on the observations.

* first guess is removed from observations
* Gridding procedure (also called "analysis"):

$$
\mbox{Gridded field} = \mbox{First guess of field} + \mbox{weighted sum of observation}
$$

There are several ways to define the
weights, which result in different gridding
techniques. We will review in the following the most used gridding
methods.

### Cressman method

Cressman weights depend only on the distance between the location
 where the value of the field should be estimated and the
location of the observation:

Cressman scheme:
$$\begin{alignat}{2}
w(d) &\;=\;& \frac{R^2 - d^2}{R^2 + d^2} \quad \mbox{if $d < R$} \\
&\;=\;& 0 \quad \mbox{if $d \ge R$}
\end{alignat}
$$

Barnes scheme:
$$
w(d) = \exp\left( - \frac{d^2}{2 R^2} \right)
$$


In [ ]:
x_coord = -4:0.1:4
# distance to observation at x=0
d = abs.(x_coord)
R = 2;
w = (R .^ 2 .- d .^ 2) ./ (R .^ 2 .+ d .^ 2);
w[d.>=R] .= 0;

fig = Figure()
ax = Axis(fig[1, 1])
lines!(ax, x_coord, w, color = :blue, label = "Cressman scheme")

R = 1;
w = exp.(-d .^ 2 ./ (2 * R .^ 2));
lines!(ax, x_coord, w, color = :red, label = "Barnes scheme")

axislegend(ax, "Weights", framevisible = true)
fig

The search radius `R` is the typical control parameter and defines the
length-scale over which an observation is used.  This length scale
can be made to vary in space depending on data coverage and/or
physical scales. This parameter is chosen by the users based on their
knowledge of the domain and the problem.


In [ ]:
include("../cressman2.jl")
R = 2.0
fi = cressman2(R, X, Y, Fe, x, y);

In [ ]:
plotfieldcmp(f2, fi, figtitle = "Gridded field by Cressman weighting (R=$(R))")

The Cressman weighting is a very simple and numerically quite
efficient method. However, it suffers from some limitations which are
apparent in the previous figure.

* No estimate can be obtained at locations when no observation is
  located within the $R$ (see blank areas in the right subplot).
* In regions with very few observations, the method can return a
  discontinuous field.
* The presence of barriers
  cannot be taken into account easily.
* All observations are assumed to have a similar error
  variance since the weighting is based only on distance.

* As a variant of the Cressman weights, other weighting functions can be defined. In the Barnes scheme, the weights are defined using a Gaussian function


In [ ]:
fi = cressman2(R / 2, X, Y, Fe, x, y; method = :barnes);

In [ ]:
plotfieldcmp(f2, fi, figtitle = "Gridded field using Barnes weights")

## Optimal interpolation

*  The first guess and the observations have errors
*  These errors are characterized by the error covariance
*  Optimal interpolation seeks the optimal combination between the first guess and the observations

### Error covariances

*  Need to be specified for observation and first guess
*  Error covariances describe __how different variables or the same
variable at different locations are related in average__. 
*  The error covariance is in general a matrix.
*  the square root of the diagonal elements is the average error of the observations and the first guess
*  the off-diagonal elements describe the relationships between the variables
*  Covariance is directly related to the correlation

### Assumptions
Hypotheses concerning those errors:
*  observations and the first guess are __unbiased__, _i.e._ the error is zero in average.
*  __error covariance matrices are known__. Those matrices are related to the expected magnitude of the error.
*  the error of the first guess is __independent__ of the observation errors.


### Analysis

The optimal interpolation (OI)
scheme can be derived as the Best Linear Unbiased Estimator (BLUE) of
the true state which has the following properties:

*  The estimator __is linear__ in the first guess and observations
*  The estimator is __not biased__
*  This estimate has a __minimal total variance__, _i.e._ no
    other estimator would have an error variance lower that the BLUE
    estimator.

From these assumptions the optimal weights of the observations can be derived to grid the field.
The optimal weights depend on the error covariances. In particular,

*  Large expected error of observations compared to first guess $\rightarrow$ more weight to the observations than the first guess
*  Variables are strongly correlated spatially $\rightarrow$ smooth gridded field.
*  See file `diva_intro_slides.pdf` if you want to know how the weights are computed
*  __An error estimate__ can also be computed by optimal interpolation.


In [ ]:
include("../OptimInterp.jl");

In [ ]:
R = 2.2 * 1.6855;
R = 3.0;
Var = fill(1 / 3, (length(Fe),));

meanF = mean(Fe)
Fa = Fe .- meanF;

fi, vari = OptimInterp.optiminterp((X, Y), Fa, Var, (R, R), length(Fa), (x, y));
fi = fi .+ meanF;

As an illustration, the field obtained by optimal interpolation is
shown

In [ ]:
plotfieldcmp(f2, fi, figtitle = "Optimal interpolation");

Relative error variance of the gridded field by optimal interpolation

In [ ]:
plotfield(vari, vmin = 0, vmax = 1.0, figtitle = "Relative error variance")

## Variational Inverse Method and DIVA/DIVAnd

The variational inverse method (VIM) or variational analysis ([Brasseur et al., 1996](https://www.sciencedirect.com/science/article/pii/096706379600012X))
 aims to find a field with the following characteristics:


* __smooth__
* somewhat __close__ to first guess
* __close__ to the observed values

This is achieved by defining a cost function which penalizes a field that doesn't satisfy those requirements.


* "__close__" is quantified using the RMS error
* "__smooth__" is quantified by averaging the  square of the first and second derivatives of the field (for a slowly varying field those derivatives are small).
* those different requirements are weighted to define what is more important, a smooth field or a field close to observations.


### Equivalence between OI and variational analysis


* The variational inverse method is identical to the optimal interpolation analysis if the weights are defined as the inverse of the covariance matrix used in optimal interpolation analysis.
* This allows to compute the weights based on the error covariances
* Error covariance can be obtained by the statistical behaviour of the field

### Error fields by analogy with optimal interpolation

Error fields can be derived since:


* The optimal interpolation analysis is equivalent to VIM if the weights used by VIM are the inverse of the error covariances used by optimal interpolation.
* In the context of optimal interpolation, it can be shown that the error field equals the analysis of the covariance fields
* $\Rightarrow$ error field of VIM equals analysis (by VIM) of covariance fields (the input of the analysis tool is a vector containing the covariance of data points with the point in which the error estimate is to be calculated)


The tools `DIVA` and `DIVAnd` implement the variational inverse method.


### Example

Field reconstructed using DIVA implementing the  variational inverse method. Right panel shows true field.


* The results are similar to the field obtained by optimal interpolation. 
* But now, the land-sea boundary is taken into account.





In [ ]:
using DIVAnd

Type ```?DIVAnd``` for the documentation of DIVAndrun.

In [ ]:
len = 2.0
epsilon2 = 1 / 3;
fi, s = DIVAndrun(mask, (pm, pn), (x, y), (X, Y), Fa, (len, len), epsilon2);

fi = fi .+ meanF;

In [ ]:
plotfieldcmp(f2, fi, figtitle = "Variational inverse method (DIVAnd)");

Relative error of the field reconstructed using DIVA


* The error is lowest near the observations and tends to 1 far away from them. 
* The presence of the barrier also impacts the estimation of error variance.
* The error estimate should be interpreted cautiously since we supposed that the used error covariances are correct.


In [ ]:
err = DIVAnd_cpme(mask, (pm, pn), (x, y), (X, Y), Fa, (len, len), epsilon2);
plotfield(err, vmin = 0, vmax = 1.0, figtitle = "DIVAnd error field");